In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from scipy.misc import imresize

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import backend as K
 
 
seed = 42
epochs = 10
records_per_class = 100
 
# We take only 2 classes from CIFAR10 and a very small sample to intentionally overfit the model.
# We will also use the same data for train/test and expect that Keras will give the same accuracy.
(x, y), _ = cifar10.load_data()
 
def filter_resize(category):
   # We do the preprocessing here instead in the Generator to get around a bug on Keras 2.1.5.
   return [preprocess_input(imresize(img, (224,224)).astype('float')) for img in x[y.flatten()==category][:records_per_class]]
 
x = np.stack(filter_resize(3)+filter_resize(5))
records_per_class = x.shape[0] // 2
y = np.array([[1,0]]*records_per_class + [[0,1]]*records_per_class)
 
 

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


In [2]:
# We will use a pre-trained model and finetune the top layers.
np.random.seed(seed)
with K.learning_phase_scope(0):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
l = Flatten()(base_model.output)
predictions = Dense(2, activation='softmax')(l)
model = Model(inputs=base_model.input, outputs=predictions)
 
for layer in model.layers[:140]:
   layer.trainable = False
 
for layer in model.layers[140:]:
   layer.trainable = True
 
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(ImageDataGenerator().flow(x, y, seed=42), epochs=epochs, validation_data=ImageDataGenerator().flow(x, y, seed=42))
 
# Store the model on disk
model.save('tmp.h5')
 
 

/root/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/10
7/7 [==============================] - 5s 709ms/step - loss: 5.8444 - accuracy: 0.5600 - val_loss: 0.1807 - val_accuracy: 0.9050
Epoch 2/10
7/7 [==============================] - 1s 155ms/step - loss: 0.1672 - accuracy: 0.9150 - val_loss: 0.0233 - val_accuracy: 1.0000
Epoch 3/10
7/7 [==============================] - 1s 156ms/step - loss: 0.0233 - accuracy: 1.0000 - val_loss: 0.0060 - val_accuracy: 1.0000
Epoch 4/10
7/7 [==============================] - 1s 154ms/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 0.0046 - val_accuracy: 1.0000
Epoch 5/10
7/7 [==============================] - 1s 156ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.0039 - val_accuracy: 1.0000
Epoch 6/10
7/7 [==============================] - 1s 154ms/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 1.0000
Epoch 7/10
7/7 [==============================] - 1s 156ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.0035 - val_accuracy: 1.0000
Epoch 8/10
7/7 [====

In [3]:
# In every test we will clear the session and reload the model to force Learning_Phase values to change.
print('DYNAMIC LEARNING_PHASE')
K.clear_session()
model = load_model('tmp.h5')
# This accuracy should match exactly the one of the validation set on the last iteration.
print(model.evaluate_generator(ImageDataGenerator().flow(x, y, seed=42)))
 
 
print('STATIC LEARNING_PHASE = 0')
K.clear_session()
K.set_learning_phase(0)
model = load_model('tmp.h5')
# Again the accuracy should match the above.
print(model.evaluate_generator(ImageDataGenerator().flow(x, y, seed=42)))
 
 
print('STATIC LEARNING_PHASE = 1')
K.clear_session()
K.set_learning_phase(1)
model = load_model('tmp.h5')
# The accuracy will be close to the one of the training set on the last iteration.
print(model.evaluate_generator(ImageDataGenerator().flow(x, y, seed=42)))

DYNAMIC LEARNING_PHASE
[0.0023754646946534713, 1.0]
STATIC LEARNING_PHASE = 0
[0.0023754646946534713, 1.0]
STATIC LEARNING_PHASE = 1
[0.23021982716662542, 0.905]
